In [1]:
from llmexp.llm.smollm import LLMWrapper
from accelerate import Accelerator

# checkpoint = "meta-llama/Meta-Llama-3-8B-Instruct"
checkpoint = "meta-llama/Llama-3.2-1B-Instruct"
# checkpoint = "HuggingFaceTB/SmolLM-1.7B-Instruct"


accelerator = Accelerator()
device = accelerator.device


llm = LLMWrapper(checkpoint, device=device)
tokenizer = llm.tokenizer
llm.hidden_size


2048

In [2]:
# instruction = "Analyze the sentiment of the following sentence. Be brief."
instruction = "Analyze the sentiment of the following sentence and respond with only one word: 'positive,' 'negative,' or 'neutral,' based on the overall tone and meaning of the sentence. Do not provide any additional explanation."
user_input = "I hate this movie."

content = [
            {"role": "system", 
            "content": instruction
            },

            {"role": "sentence", 
            "content": user_input
            }
        ]
template = tokenizer.apply_chat_template(content, tokenize=False, add_generation_prompt=True)
# print(template)

# The generated outputs 
gen_output = llm.generate_from_texts(template)
print(gen_output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond with only one word: 'positive,' 'negative,' or 'neutral,' based on the overall tone and meaning of the sentence. Do not provide any additional explanation.<|eot_id|><|start_header_id|>sentence<|end_header_id|>

I hate this movie.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

negative<|eot_id|>


In [3]:
from llmexp.utils.data_utils import LLMDataset, create_dataloader
dataloader = create_dataloader('sst2', tokenizer, instruction=instruction)
# batch = collate_fun(dataset[:10])
inputs = next(iter(dataloader)).to(device)
inputs

{'input_ids': tensor([[128009, 128009, 128009,  ...,  78191, 128007,    271],
        [128009, 128009, 128009,  ...,  78191, 128007,    271],
        [128009, 128009, 128009,  ...,  78191, 128007,    271],
        ...,
        [128009, 128009, 128009,  ...,  78191, 128007,    271],
        [128009, 128009, 128009,  ...,  78191, 128007,    271],
        [128009, 128009, 128009,  ...,  78191, 128007,    271]],
       device='cuda:0'), 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]], device='cuda:0'), 'labels': tensor([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1,
        0, 0, 1, 0, 0, 1, 1, 1], device='cuda:0')}

In [4]:
generated_outputs = llm.generate(inputs,  max_new_tokens=50)
generated_outputs

{'input_ids': tensor([[128009, 128009, 128009,  ...,    271,  36590, 128009],
         [128009, 128009, 128009,  ...,    271,  31587, 128009],
         [128009, 128009, 128009,  ...,    271,  31587, 128009],
         ...,
         [128009, 128009, 128009,  ...,    271,  31587, 128009],
         [128009, 128009, 128009,  ...,    271,  31587, 128009],
         [128009, 128009, 128009,  ...,    271,  31587, 128009]],
        device='cuda:0'),
 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 0],
         [0, 0, 0,  ..., 1, 1, 0],
         [0, 0, 0,  ..., 1, 1, 0],
         ...,
         [0, 0, 0,  ..., 1, 1, 0],
         [0, 0, 0,  ..., 1, 1, 0],
         [0, 0, 0,  ..., 1, 1, 0]], device='cuda:0')}

In [5]:
from llmexp.utils import decode_batch_outputs

decoded_outputs = decode_batch_outputs(**generated_outputs, tokenizer=tokenizer)
print(decoded_outputs[1])

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond with only one word: 'positive,' 'negative,' or 'neutral,' based on the overall tone and meaning of the sentence. Do not provide any additional explanation.<|eot_id|><|start_header_id|>user<|end_header_id|>

a minor miracle<|eot_id|><|start_header_id|>assistant<|end_header_id|>

positive


In [8]:
output_logits = llm(**generated_outputs).logits
print(output_logits)
print(output_logits.shape)

tensor([[[ -6.0502,  -9.7437,  -8.4180,  ...,   9.6519,   9.6512,   9.6513],
         [ -7.5367,  -9.8883,  -7.2537,  ...,   9.1260,   9.1252,   9.1252],
         [ -7.0737,  -9.6388,  -6.2391,  ...,   9.5185,   9.5179,   9.5182],
         ...,
         [  7.4884,  16.6444,  12.5971,  ...,   3.8764,   3.8753,   3.8754],
         [ 12.2231,  10.0633,   1.0912,  ...,   2.6447,   2.6436,   2.6442],
         [ -1.7467,  -4.1949,  -4.9134,  ...,   8.8656,   8.8649,   8.8649]],

        [[ -6.4945, -10.2150,  -8.4819,  ...,   9.8243,   9.8233,   9.8233],
         [ -8.9331, -11.6798,  -8.0162,  ...,   9.9392,   9.9385,   9.9382],
         [ -7.6666, -10.0353,  -6.7793,  ...,  10.0226,  10.0221,  10.0221],
         ...,
         [  8.1064,  15.2264,  13.7822,  ...,   5.4619,   5.4608,   5.4613],
         [ 11.7456,  10.9452,   1.7162,  ...,   3.1175,   3.1164,   3.1170],
         [ -2.6772,  -7.5897,  -5.3533,  ...,   9.3135,   9.3124,   9.3125]],

        [[ -6.5032,  -8.3647,  -8.1158,  ...

In [17]:
import torch
torch.argmax(output_logits[0,20])

tensor(128009, device='cuda:0')

In [7]:
print(inputs)
print(generated_outputs)

{'input_ids': tensor([[128009, 128009, 128009,  ...,  78191, 128007,    271],
        [128009, 128009, 128009,  ...,  78191, 128007,    271],
        [128009, 128009, 128009,  ...,  78191, 128007,    271],
        ...,
        [128009, 128009, 128009,  ...,  78191, 128007,    271],
        [128009, 128009, 128009,  ...,  78191, 128007,    271],
        [128009, 128009, 128009,  ...,  78191, 128007,    271]],
       device='cuda:0'), 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]], device='cuda:0'), 'labels': tensor([1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1,
        0, 1, 0, 0, 1, 0, 0, 1], device='cuda:0')}
{'input_ids': tensor([[128009, 128009, 128009,  ..., 128009, 128009, 128009],
        [128009, 128009, 128009,  ..., 128009, 128009, 128009],
        [128009, 128009, 1

In [8]:
from llmexp.trainer.mab_trainer import randomly_cut_and_pad_generations
random_cropped_inputs =randomly_cut_and_pad_generations(inputs, generated_outputs, tokenizer)

In [9]:
from llmexp.explainer.mab_model import MABModel
mab_model = MABModel(llm, hidden_size=2048).to(device)
mu_logits = mab_model.forward(**random_cropped_inputs)
mu_logits

tensor([[-0.0263, -0.0066, -0.0134,  ...,  0.0087, -0.0066,  0.0210],
        [-0.0017,  0.0019, -0.0087,  ..., -0.0011,  0.0053,  0.0127],
        [-0.0095, -0.0140, -0.0140,  ...,  0.0107, -0.0203,  0.0189],
        ...,
        [-0.0130, -0.0130, -0.0140,  ..., -0.0132, -0.0129,  0.0084],
        [ 0.0066,  0.0051,  0.0033,  ...,  0.0083, -0.0102,  0.0258],
        [-0.0003, -0.0062, -0.0081,  ..., -0.0084, -0.0175,  0.0106]],
       device='cuda:0', grad_fn=<SqueezeBackward1>)

In [10]:
mu_logits.shape

torch.Size([32, 119])

In [12]:
random_cropped_inputs['input_ids'].shape

torch.Size([32, 120])

In [43]:
tokenizer.decode(torch.argmax(output_logits[0,53]))

' text'

In [58]:
torch.max(torch.softmax(output_logits[0,19], dim=-1), dim=-1)

torch.return_types.max(
values=tensor(0.7496, device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor(128009, device='cuda:0'))

In [46]:
tokenizer.decode(inputs['input_ids'][0,54])

' following'